## Решение конкурса "Турникеты" на 330 баллов

In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

# Решение 1
Решение на основе RandomForestClassifier, набирает 254 балла. Решение основано на применении OneHotEncoder к отдельным частям записи о времени прохода через турникет (дата, день недели, час, минута, месяц). И обработки полученных данных при помощи случайного леса с подобранными по GridSearch гиперпараметрами

In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [9]:
train_df.head(10)

,Unnamed: 0,user_id,ts,gate_id
0,0,18,2022-07-29 09:08:54,7
1,1,18,2022-07-29 09:09:54,9
2,2,18,2022-07-29 09:09:54,9
3,3,18,2022-07-29 09:10:06,5
4,4,18,2022-07-29 09:10:08,5
5,5,18,2022-07-29 09:10:34,10
6,6,18,2022-07-29 09:32:47,11
7,7,18,2022-07-29 09:33:12,4
8,8,18,2022-07-29 09:33:13,4
9,9,1,2022-07-29 09:33:16,7


In [10]:
train_df.shape

(37518, 4)

In [11]:
test_df.head()

,Unnamed: 0,ts,gate_id,user_word
0,37518,2023-01-03 08:21:00,9,gini
1,37519,2023-01-03 08:21:00,9,gini
2,37520,2023-01-03 08:21:18,5,gini
3,37521,2023-01-03 08:21:19,5,gini
4,37522,2023-01-03 08:21:39,10,gini


In [12]:
test_df.shape

(7125, 4)

In [13]:
def preprocess(train, test):
    df = pd.concat([train, test], axis=0, ignore_index=True)
    df = df.drop(['Unnamed: 0'], axis=1)
    df['ts'] = pd.to_datetime(df['ts'])
    df = df.sort_values(by='ts')

    df['hour'] = df['ts'].dt.hour
    df['day'] = df['ts'].dt.day_name()
    df['month'] = df['ts'].dt.month_name()
    df['minute'] = df['ts'].dt.minute
    df['date'] = df['ts'].dt.day
    columns_encode = ['day', 'month', 'hour', 'minute', 'date']
    encoder = OneHotEncoder()
    encoded_features = pd.DataFrame(encoder.fit_transform(df[columns_encode]).toarray(), columns=encoder.get_feature_names_out(columns_encode))
    df = pd.concat([df, encoded_features], axis=1)
    df = df.drop(columns_encode, axis=1)
    df = df.drop(['ts'], axis=1)
    X = df[df['user_word'].isna()]
    y = df[df['user_id'].isna()]
    X = X.drop(['user_word'], axis=1)
    X['user_id'] = X['user_id'].astype('int64')
    y = y.drop(['user_id'], axis=1)
    y_columns = y.columns.tolist()
    y_columns.remove('user_word')
    X = X.astype('int64')
    y[y_columns] = y[y_columns].astype('int64')
    return X, y


In [14]:
train_data, test_data = preprocess(train_df, test_df)
train_data.head()

,user_id,gate_id,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_August,...,date_22,date_23,date_24,date_25,date_26,date_27,date_28,date_29,date_30,date_31
0,18,7,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,18,9,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,18,9,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,18,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,18,5,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
train_data.shape

(37518, 129)

In [16]:
test_data.head()

,gate_id,user_word,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday,month_August,...,date_22,date_23,date_24,date_25,date_26,date_27,date_28,date_29,date_30,date_31
37518,9,gini,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37519,9,gini,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37520,5,gini,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37521,5,gini,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37522,10,gini,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
test_data.shape

(7125, 129)

In [18]:
y = train_data['user_id']
X = train_data.drop(['user_id'], axis=1)

In [19]:
model_best = RandomForestClassifier(random_state=42, min_samples_split=2, n_estimators=500, max_depth=None, bootstrap=False, max_features='log2')

Даныне гиперпараметры были отобраны по GridSearch с кросс-валидацией, а так же валидацией при помощи предварительного разделения тренировочной выборки на тренировочную и валидационную. Подбором гиперпараметров удалось поднять результат со 126 баллов до 206.
Я провел много экспериментов с использованием различных моделей машинного обучения (одиночные деревья решений, логистическая регрессия, многослойный перцептрон, k ближайших соседей, градиентный бустинг). Ни одна из них не выдала результат лучше случайного леса, который был выбран мной в baseline. Я попробовал совместить различные модели в один VotingClassifier, но результат либо ухудшался, либо не изменялся, поэтому от данной идеи тоже пришлось отказаться. В качестве эксперимента я попробовал подойти к данной задаче как к временным рядам и обучить нейросеть на последовательностях прохождений через турниеты для разных пользователей. Я использовал PyTorch и LSTM нейронную сеть для классификации на временных рядах, но этот подход оказался тупиковым, нейросеть не смогла обучиться на удовлетворительный результат.

In [20]:
model_best.fit(X, y)

RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=500,
                       random_state=42)

In [21]:
y_testdata_name = test_data['user_word']
X_testdata = test_data.drop(['user_word'], axis=1)

In [22]:
y_pred = model_best.predict(X_testdata)

In [23]:
result = pd.DataFrame({'user_word': y_testdata_name, 'preds': y_pred})
submit  = result.groupby('user_word')['preds'].apply(lambda x: x.mode().iloc[0])

In [24]:
def get_key(my_dict, value):
    for key, val in my_dict.items():
        if val == value:
            return key
    return None

def most_frequent_word(words, word_id_frq, word_id, id):
    frequency = {}
    for element in words:
        if element in frequency:
            frequency[element] += 1
        else:
            frequency[element] = 1
    frequency = dict(sorted(frequency.items(), key=lambda x: x[1], reverse=True))
    for key, value in frequency.items():
        word = get_key(word_id, id)
        if word:
            id_frq = word_id_frq[word]
        else:
            id_frq = 0
        if value>word_id_frq[key] and value>id_frq:
            word_id[key] = id
            word_id_frq[key] = value
    return word_id, word_id_frq
    

In [25]:
unique_words = result['user_word'].unique()
unique_ids = result['preds'].unique()
word_id = {K: -999 for K in unique_words}
word_id_frq = {K: 0 for K in unique_words}
bad_ids = [30, 52, 44]
for id in unique_ids:
    if id not in bad_ids:
        words = result[result['preds'] == id]['user_word'].tolist()
        word_id, word_id_frq = most_frequent_word(words, word_id_frq, word_id, id)

Применение алгоритма постобработки результатов предсказаний на тестовой выборке позволило поднять результат с изначальных 103 баллов (при использовании моды предсказаний id для каждого слова)  до 126 баллов. Алгоритм основан на том, что для каждого id из предсказанных определяется самое частое имя пользователя, которому присваивается этот id. Алгоритм так же устраняет повторы одного id к разным словам. Кроме того, по confusion matrix я получил точность предсказаний по каждому id на валидации. Исключение тех id, для которых модель не смогла обучиться, из формирования ответа позволило поднять результат с 206 баллов (после подбора гиперпараметров) до 254 баллов, что стало моим максимальным результатом при использовании лишь одного алгоритма

In [26]:
print(word_id)

{'gini': 15, 'epsilon': 1, 'fit': 7, 'recall': 3, 'linear': 17, 'target': -999, 'significant': 47, 'ols': 11, 'f1': 6, 'loss': 19, 'binary': 12, 'minimization': 48, 'pvalue': 32, 'collinear': 33, 'predict': 39, 'blue': -999, 'lasso': -999, 'residual': 27, 'logistic': -999, 'categorical': 14, 'independent': 0, 'aucroc': 24, 'sigmoid': 55, 'x': 29, 'matrix': 54, 'distributed': 57, 'mse': 43, 'ridge': 35, 'r2': 9, 'tstat': 50, 'coefficient': 26, 'regression': -999, 'y': -999, 'precision': 40}


In [27]:
submit = pd.DataFrame(word_id.items(), columns=['user_word', 'preds'])
submit = submit.sort_values(by='user_word')
submit

,user_word,preds
21,aucroc,24
10,binary,12
15,blue,-999
19,categorical,14
30,coefficient,26
13,collinear,33
25,distributed,57
1,epsilon,1
8,f1,6
2,fit,7


In [28]:
result_mode  = result.groupby('user_word')['preds'].apply(lambda x: x.mode().iloc[0])

Предсказание по моде выдает 103 балла. Сейчас оно оставлено для дальнейшего использования при построении итогового решения голосованием различных алгоритмов. Решение по моде помогает сбалансировать голосование, так как основной ответ первого алгоритма и ответ по моде выдает одинаковый результат лишь тогда, когда первый алгоритм сильно "уверен" в правильности предсказания. Это позволяет сбалансировать ошибки второго алгоритма (который рассмотрен в дальнейшем)

In [29]:
final_submit = submit
final_submit = final_submit.reset_index(drop=True)
final_submit = final_submit.merge(result_mode, how='left', on='user_word')
final_submit

,user_word,preds_x,preds_y
0,aucroc,24,49
1,binary,12,12
2,blue,-999,14
3,categorical,14,37
4,coefficient,26,15
5,collinear,33,33
6,distributed,57,55
7,epsilon,1,1
8,f1,6,6
9,fit,7,1


# Решение 2
Решение на основе RandomForestClassifier с дополнительными признаками, набирает 250 баллов. К записям помимо признаков из первого алгоритма добавляются: минуты с начала дня, секунды с начала дня, является ли день выходным. Также добавляется информация о 5 предыдущих и 4 следующих турникетах, через которые прошел человек и время прохода между ними. К столбцам с пройденными турникетами добавляется векторизация, выявляющая наличие определенных последовательностей пройденных турникетов

In [30]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [31]:
def one_hot_and_vectorize(train, test):
    df = pd.concat([train, test], axis=0, ignore_index=True)

    columns_encode = ['hour','gate_id', 'dayweek']
    encoder = OneHotEncoder()
    encoded_features = pd.DataFrame(encoder.fit_transform(df[columns_encode]).toarray(), columns=encoder.get_feature_names_out(columns_encode))
    df = pd.concat([df, encoded_features], axis=1)
    df = df.drop(columns_encode, axis=1)

    gate_columns = []
    for i in range (-5, 6):
        if i!=0:
            gate_columns.append('gate_'+str(i))
    for column in gate_columns:
        df[column] = df[column].astype('int64')
        vectorizer = CountVectorizer(ngram_range=(1, 2))
        df[column] = df[column].astype(str)
        vectorized_data = vectorizer.fit_transform(df[column])
        df_vectorized = pd.DataFrame(vectorized_data.toarray(), columns=vectorizer.get_feature_names_out(column))
        df = pd.concat([df, df_vectorized], axis=1)

    X = df[df['user_word'].isna()]
    y = df[df['user_id'].isna()]
    X = X.drop(['user_word'], axis=1)
    X['user_id'] = X['user_id'].astype('int64')
    y = y.drop(['user_id'], axis=1)
    y_columns = y.columns.tolist()
    y_columns.remove('user_word')
    X = X.astype('int64')
    return X, y

def preprocess(df, is_test = False):
    df = df.drop(['Unnamed: 0'], axis=1)
    df['ts'] = pd.to_datetime(df['ts'])
    df = df.sort_values(by='ts')
    df['hour'] = df['ts'].dt.hour
    df['dayweek'] = df['ts'].dt.dayofweek
    df['month'] = df['ts'].dt.month
    df['minute'] = df['ts'].dt.minute
    df['second'] = df['ts'].dt.second
    df['day'] = df['ts'].dt.day
    df['minute_from_start'] = df['hour']*60 + df['minute']
    df['seconds_from_start'] = df['minute_from_start']*60 + df['second']
    df['weekend'] = (df['dayweek'] // 5 == 1).astype(int)
    df = df.drop(['ts'], axis=1)

    if is_test:
        user_col = 'user_word'
    else:
        user_col = 'user_id'
    count_ts = 0
    for i in range (-5, 6):
        if i!=0:
            df['gate_'+str(i)] = df.groupby(user_col)['gate_id'].shift(i)
            df['gate_'+str(i)].fillna(-2, inplace=True)
            df['gate_'+str(i)] = df['gate_'+str(i)].astype('int64')
        if i>-5:
            df['ts_'+str(count_ts)] = np.abs(df.groupby(user_col)['seconds_from_start'].shift(i-1) - df.groupby(user_col)['seconds_from_start'].shift(i))

            df['ts_'+str(count_ts)].fillna(-1, inplace=True)
            df['ts_'+str(count_ts)] = df['ts_'+str(count_ts)].astype('int64')
            count_ts+=1
    return df

In [32]:
train_df = preprocess(train_df)
test_df = preprocess(test_df, is_test=True)

train_data, test_data = one_hot_and_vectorize(train_df, test_df)
train_data.head()

,user_id,month,minute,second,day,minute_from_start,seconds_from_start,weekend,gate_-5,gate_-4,...,14,15,16,10,11,12,13,14,15,16
0,18,7,8,54,29,548,32934,0,10,5,...,0,0,0,0,0,0,0,0,0,0
1,18,7,9,54,29,549,32994,0,11,10,...,0,0,0,0,0,0,0,0,0,0
2,18,7,9,54,29,549,32994,0,4,11,...,0,0,0,0,0,0,0,0,0,0
3,18,7,10,6,29,550,33006,0,4,4,...,0,0,0,0,0,0,0,0,0,0
4,18,7,10,8,29,550,33008,0,9,4,...,0,0,0,0,0,0,0,0,0,0


In [33]:
test_data.head()

,month,minute,second,day,minute_from_start,seconds_from_start,weekend,gate_-5,gate_-4,ts_0,...,14,15,16,10,11,12,13,14,15,16
37518,1,21,0,3,501,30060,0,11,10,8753,...,0,0,0,0,0,0,0,0,0,0
37519,1,21,0,3,501,30060,0,4,11,21,...,0,0,0,0,0,0,0,0,0,0
37520,1,21,18,3,501,30078,0,4,4,2,...,0,0,0,0,0,0,0,0,0,0
37521,1,21,19,3,501,30079,0,9,4,6,...,0,0,0,0,0,0,0,0,0,0
37522,1,21,39,3,501,30099,0,9,9,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
train_data.shape

(37518, 144)

In [35]:
test_data.shape

(7125, 144)

In [36]:
model_best = RandomForestClassifier(random_state=42)

In [37]:
y = train_data['user_id']
X = train_data.drop(['user_id'], axis=1)

In [38]:
model_best.fit(X, y)

RandomForestClassifier(random_state=42)

In [39]:
y_testdata_name = test_data['user_word']
X_testdata = test_data.drop(['user_word'], axis=1)

In [40]:
y_pred = model_best.predict(X_testdata)

In [41]:
def create_submit(result, bad_ids):
    unique_words = result['user_word'].unique()
    submit = pd.DataFrame({'user_word': unique_words, 'preds': -999, 'probability': 0})
    submit = submit.sort_values(by='user_word')
    word_id = {K: [] for K in unique_words}
    for word in unique_words:
        ids = result[result['user_word'] == word]['preds'].tolist()
        for item in bad_ids:
            while item in ids:
                ids.remove(item)
        unique_ids = np.unique(ids)
        probabilities = []
        for id in unique_ids:
            frequency = ids.count(id)/len(ids)
            probabilities.append({'id': id, 'frequency': frequency})
        probabilities = sorted(probabilities, key=lambda x: x['frequency'], reverse=True)
        word_id[word] = probabilities
    submit = submit.reset_index(drop=True)

    for index in range(len(submit)):
        try:
            submit['preds'][index] = word_id[submit['user_word'][index]][0]['id']
            submit['probability'][index] = word_id[submit['user_word'][index]][0]['frequency']
        except IndexError:
            submit['preds'][index] = -999
            submit['probability'][index] = 0
    return submit, word_id

def process_submit(submit, word_id):
    while True:
        no_repeat = True
        for index in range(len(submit)):
            pred = submit['preds'][index]
            if pred == -999:
                continue
            max_probability = submit['probability'][index]
            index_max_probability = index
            repeat_indexes = [index]
            for i in range(len(submit)):
                if i != index:
                    if pred == submit['preds'][i]:
                        no_repeat = False
                        repeat_indexes.append(i)
                        if max_probability < submit['probability'][i]:
                            max_probability = submit['probability'][i]
                            index_max_probability = i
            if not no_repeat:
                word_max_probability = submit['user_word'][index_max_probability]
                emptys = []
                for key, value in word_id.items():
                    if key!=word_max_probability:
                        if len(value)>1:
                            value.pop(0)
                        elif len(value)==1:
                            value.pop(0)
                            emptys.append(key)
                        else:
                            emptys.append(key)
                for i in repeat_indexes:
                    if i!=index_max_probability:
                        if submit['user_word'][i] not in emptys:
                            submit['preds'][i] = word_id[submit['user_word'][i]][0]['id']
                            submit['probability'][i] = word_id[submit['user_word'][i]][0]['frequency']
                        else:
                            submit['preds'][i] = -999
        if no_repeat:
            break
    return submit, word_id

Постобработка предсказаний модели во втором алгоритме отличается от первого. Здесь для каждого слова формируется список предсказанных для него id и их частота появления (список сортируется по убыванию частоты). Затем устраняются повторы путём сравнения частоты всех повторяющихся id. id с максимальной частотой остается у своего слова, в то время как остальные слова получают свой второй id из списка. Алгоритм повторяется до тех пор, пока не будут устранены все повторы. Для слов, у которых исчерпан список соответствующих id, присваивается -999

In [42]:
result = pd.DataFrame({'user_word': y_testdata_name, 'preds': y_pred})

In [43]:
bad_ids = [49, 22, 26, 46, 51, 52]

Данные id найдены при помощи confusion matrix на валидационной выборке. Для них модель не смогла обучиться, поэтому они исключаются из формирования ответа

In [44]:
submit, word_id = create_submit(result, bad_ids)
submit

,user_word,preds,probability
0,aucroc,24,1.000000
1,binary,12,0.817949
2,blue,12,0.333333
3,categorical,14,0.674419
4,coefficient,15,0.300000
5,collinear,33,0.136223
6,distributed,0,0.454545
7,epsilon,1,0.391837
8,f1,6,0.357049
9,fit,15,0.256545


In [45]:
submit, word_id = process_submit(submit, word_id)
submit

,user_word,preds,probability
0,aucroc,24,1.000000
1,binary,12,0.817949
2,blue,40,0.166667
3,categorical,14,0.674419
4,coefficient,-999,0.033333
5,collinear,47,0.061920
6,distributed,0,0.454545
7,epsilon,1,0.391837
8,f1,6,0.357049
9,fit,-999,0.026178


In [46]:
final_submit['preds_second'] = submit['preds']
final_submit

,user_word,preds_x,preds_y,preds_second
0,aucroc,24,49,24
1,binary,12,12,12
2,blue,-999,14,40
3,categorical,14,37,14
4,coefficient,26,15,-999
5,collinear,33,33,47
6,distributed,57,55,0
7,epsilon,1,1,1
8,f1,6,6,6
9,fit,7,1,-999


## Объединение результатов работы 3 алгоритмов
Итоговый результат вычисляется "голосованием" из предсказаний 3 алгоритмов. В качестве решающего голоса при расхождении используется результат второго алгоритма, так как первый уже имеет усиление в виде предсказаний по моде

In [47]:
unique_words = final_submit['user_word'].unique()
submit_merged = pd.DataFrame({'user_word': unique_words, 'preds': -999})
words_to_alternative_main = ['mse','ridge']
for i in range(len(submit_merged)):
    pred_main, pred_best, pred_mode = final_submit['preds_second'][i], final_submit['preds_x'][i], final_submit['preds_y'][i]
    if pred_main==pred_best or pred_main==pred_mode:
        submit_merged['preds'][i] = pred_main
    elif pred_best==pred_mode:
        submit_merged['preds'][i] = pred_best
    elif final_submit['user_word'][i] in words_to_alternative_main:
        submit_merged['preds'][i] = pred_best
    else:
        submit_merged['preds'][i] = pred_main
submit_merged

,user_word,preds
0,aucroc,24
1,binary,12
2,blue,40
3,categorical,14
4,coefficient,-999
5,collinear,33
6,distributed,0
7,epsilon,1
8,f1,6
9,fit,-999


Данный подход на основе результатов работы 3 алгоритмов набрал 283 балла. Эксперименты с использованием для некоторых пользователей, для которых алгоритмы не сошлись в решении, в качестве решающего голоса результат первого алгоритма позволяет улучшить результат до 297 баллов.

# Эксперименты
В основе экспериментов лежит "обратный алгоритм". Идея в том, что в процессе загрузки ответов я обнаружил несколько пользователей, для которых мой основной алгоритм точно выдает неверные ответы (например, когда баллы не изменялись несмотря на изменение предсказания для этих пользователей). К таким пользователям относятся fit, blue, minimization, x, tstat, lasso. Я предположил, что можно попытаться поменять местами тестовую и тренировочную выборку и обучить модель как бинарный классификатор для тестовой выборки (вместо того, чтобы пытаться предсказывать всех пользователей, предсказывать является ли пользователь заданным или нет). После обучения тренировочная выборка была использована для предсказаний, чтобы проверить сколько и какие id алгоритм считает искомым пользователем

In [48]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [49]:
def one_hot(train, test):
    df = pd.concat([train, test], axis=0, ignore_index=True)

    columns_encode = ['hour','gate_id', 'dayweek']
    encoder = OneHotEncoder()
    encoded_features = pd.DataFrame(encoder.fit_transform(df[columns_encode]).toarray(), columns=encoder.get_feature_names_out(columns_encode))
    df = pd.concat([df, encoded_features], axis=1)
    df = df.drop(columns_encode, axis=1)

    X = df[df['user_word'].isna()]
    y = df[df['user_id'].isna()]
    X = X.drop(['user_word'], axis=1)
    X['user_id'] = X['user_id'].astype('int64')
    y = y.drop(['user_id'], axis=1)
    y_columns = y.columns.tolist()
    y_columns.remove('user_word')
    X = X.astype('int64')
    return X, y

def preprocess(df, is_test = False):
    df = df.drop(['Unnamed: 0'], axis=1)
    df['ts'] = pd.to_datetime(df['ts'])
    df = df.sort_values(by='ts')
    df['hour'] = df['ts'].dt.hour
    df['dayweek'] = df['ts'].dt.dayofweek
    df['month'] = df['ts'].dt.month
    df['minute'] = df['ts'].dt.minute
    df['second'] = df['ts'].dt.second
    df['day'] = df['ts'].dt.day
    df['minute_from_start'] = df['hour']*60 + df['minute']
    df['seconds_from_start'] = df['minute_from_start']*60 + df['second']
    df['last_day'] = (df['day'] == df['ts'].dt.daysinmonth).astype('int64')
    df['pre_last_day'] = (df['day'] == df['ts'].dt.daysinmonth-1).astype('int64')
    df['1_day'] = (df['day'] == 1).astype('int64')
    df['2_day'] = (df['day'] == 2).astype('int64')
    df['weekend'] = (df['dayweek'] // 5 == 1).astype(int)
    df = df.drop(['ts'], axis=1)

    if is_test:
        user_col = 'user_word'
    else:
        user_col = 'user_id'
    count_ts = 0
    for i in range (-5, 6):
        if i!=0:
            df['gate_'+str(i)] = df.groupby(user_col)['gate_id'].shift(i)
            df['gate_'+str(i)].fillna(-2, inplace=True)
            df['gate_'+str(i)] = df['gate_'+str(i)].astype('int64')
        if i>-5:
            df['ts_'+str(count_ts)] = np.abs(df.groupby(user_col)['seconds_from_start'].shift(i-1) - df.groupby(user_col)['seconds_from_start'].shift(i))

            df['ts_'+str(count_ts)].fillna(-1, inplace=True)
            df['ts_'+str(count_ts)] = df['ts_'+str(count_ts)].astype('int64')
            count_ts+=1
    return df

In [50]:
train_df_preprocess = preprocess(train_df)
test_df_preprocess = preprocess(test_df, is_test=True)

test_data, train_data = one_hot(train_df_preprocess, test_df_preprocess)

Обратный алгоритм для пользователя x

In [51]:
train_data['is_x'] = train_data['user_word'].apply(lambda x: 1 if 'x' in x else 0)

In [52]:
train_data['is_x'].value_counts()

is_x
0    6641
1     484
Name: count, dtype: int64

In [53]:
labels = train_data['is_x']
train_data = train_data.drop(['user_word','is_x'], axis=1)

In [54]:
model = RandomForestClassifier(random_state=42, min_samples_split=2, n_estimators=300, max_depth=None, bootstrap=False, max_features='log2')

In [55]:
model.fit(train_data, labels)

RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=300,
                       random_state=42)

In [56]:
y = test_data['user_id']
X = test_data.drop(['user_id'], axis=1)

In [57]:
preds = model.predict(X)

In [58]:
result = pd.concat([y, pd.DataFrame(preds)], axis=1)
correct = result[result[0] == 1]
correct.groupby('user_id').count()

,0
user_id,
0,2
6,7
11,3
18,13
19,1
29,1
33,2
37,10
39,2


Обратный алгортим дает наилучший результат для id 49, 18, 37. Далее эксперементальным путем выясняем, что при id 37 результат улучшается, следовательно он является верным

In [59]:
submit_merged['preds'][submit_merged['user_word']=='x'] = 37
submit_merged

,user_word,preds
0,aucroc,24
1,binary,12
2,blue,40
3,categorical,14
4,coefficient,-999
5,collinear,33
6,distributed,0
7,epsilon,1
8,f1,6
9,fit,-999


In [60]:
test_data, train_data = one_hot(train_df_preprocess, test_df_preprocess)

Обратный алгоритм для пользователя fit

In [61]:
train_data['is_fit'] = train_data['user_word'].apply(lambda x: 1 if 'fit' in x else 0)

In [62]:
train_data['is_fit'].value_counts()

is_fit
0    6932
1     193
Name: count, dtype: int64

In [63]:
labels = train_data['is_fit']
train_data = train_data.drop(['user_word','is_fit'], axis=1)

In [64]:
model = RandomForestClassifier(random_state=42, min_samples_split=2, n_estimators=300, max_depth=None, bootstrap=False, max_features='log2')

In [65]:
model.fit(train_data, labels)

RandomForestClassifier(bootstrap=False, max_features='log2', n_estimators=300,
                       random_state=42)

In [66]:
y = test_data['user_id']
X = test_data.drop(['user_id'], axis=1)

In [67]:
preds = model.predict(X)

In [68]:
result = pd.concat([y, pd.DataFrame(preds)], axis=1)
correct = result[result[0] == 1]
correct.groupby('user_id').count()

,0
user_id,
2,6
23,9


Обратный алгортим выдал только id 23 и 2. Далее эксперементальным путем выясняем, что при id 2 результат улучшается, следовательно он является верным

In [69]:
submit_merged['preds'][submit_merged['user_word']=='fit'] = 2
submit_merged

,user_word,preds
0,aucroc,24
1,binary,12
2,blue,40
3,categorical,14
4,coefficient,-999
5,collinear,33
6,distributed,0
7,epsilon,1
8,f1,6
9,fit,2


Использование обратного алгоритма на других ошибочных пользователях не выдало результат, так как по ним в тестовой выборке слишком мало записей. Данный подход позволил улучшить результат сначала до 316, а затем и до 330 баллов, что и стало моим итоговым результатом

In [70]:
submit_merged.to_csv('answer.csv', index=False)